In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from walks import Route, Point, Segment

In [ ]:
import sqlite3
from collections import defaultdict


In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cartopy.io.img_tiles as cimgt

from shapely.geometry import shape

In [ ]:
conn = sqlite3.connect('mwalks.sqlite')

points = conn.execute("""
    SELECT
        point_id
    ,   route_id
    ,   Cast(strftime('%H', dt) AS INTEGER) * 3600
      + Cast(strftime('%M', dt) AS INTEGER) * 60
      + Cast(strftime('%f', dt) AS REAL) AS secs
    ,   lat
    ,   lon
    ,   ele
    FROM points ORDER BY route_id, point_id
""").fetchall()
print(len(points))

route_bounds = conn.execute('SELECT Min(lon), Min(lat), Max(lon), Max(lat) FROM points').fetchone()
print(route_bounds)

conn.close()

In [ ]:
requested_map = cimgt.OSM()  # Open Street Map

In [ ]:
routes = defaultdict(Route)
for pt in [Point(*p) for p in points]:
    routes[pt.route_id].add_point(pt)
for route_id in range(len(routes) - 5, len(routes) + 1):
    rt = routes[route_id]
    print(f'Route {route_id}: {len(rt.points)} points, distance: {rt.metres() / 1000:.3f}km = {rt.miles():.2f}mi, {rt.time():.2f} sec = {rt.avg_mph():.2f} mph')    


In [ ]:
fig, ax = plt.subplots(figsize=(9, 13), subplot_kw=dict(projection=requested_map.crs))
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = route_bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

ax.set_extent(extent)

img = ax.add_image(requested_map, 14)

mins = 15

for rt in routes.values():
    pt = rt.point_at(mins * 60)
    plt.plot(pt.lon, pt.lat, markersize=10, marker='o', color='red')

plt.title(f'After {mins} minutes')

plt.show()